# Inverted Pendulum

My goal for this project is to explore control systems for an inverted pendulum. The basis setup is we have a cart of mass $M$ and mass at the top of a pole on the care of mass $m$. We will assume the pole itself to be massless. 

Let $\theta$ represent the angle between vertical and the pole, let $v_1$ represent the velocity of the cart and $v_2$ the velocity of the pole mass. Let $F$ be the force exerted on the cart (which we constraint to move along the x-axis) and the pole will be of length $l$. 

### Calculate the Lagrangian

By Hamilton's principle the path of the pole mass and the car minimize the functional 

$$
J[x, \theta] = \int_{t_0}^{t_1} L 
$$

Where $L$ is the Lagrangian $L = T - U$ which represents the difference in kinetic and potential energy of the system. The position of the cart at time $t$ is $(x(t), 0)$ and the position of the pole mass is $(x + l \sin{\theta}, l\cos{\theta})$ we can express the kinetic energy of the system as: 

$$ 

\begin{aligned}
T &= \frac{1}{2}M v_1^2 + \frac{1}{2} m v_2^2 \\

T &= \frac{1}{2}(M) \dot{x}^2 + \frac{1}{2}m((\dot{x} + l \dot{\theta} \cos{\theta})^2 + (-l\dot{\theta} \sin{\theta})^2) \\

T &= \frac{1}{2} (m + M) \dot{x}^2 + \frac{1}{2} m l^2 \dot{\theta}^2 - m l \dot{\theta} \dot{x} \cos{theta}

\end{aligned}
$$

And the potential energy can be expressed simply: 

$$
\begin{aligned}
U &= m g l \cos{\theta} \\
\end{aligned}
$$

### Apply the principle of least action to calculate the differential equations
By applying [D'Alambert's principle](https://en.wikipedia.org/wiki/D%27Alembert%27s_principle#:~:text=D'Alembert's%20principle%20generalizes%20the,system%2C%20result%20in%20dynamic%20equilibrium.&text=D'Alembert's%20principle%20can%20be,constraints%20that%20depend%20on%20velocities.), the euler-lagrange equations now include the non-conservative force F. 


$$
\begin{aligned}
\frac{\partial L}{\partial x} - \frac{d}{dt} \frac{\partial L}{\partial \dot{x}} &= F \\

\frac{\partial L }{\partial \theta} -  \frac{d}{dt}  \frac{\partial L}{\partial \dot{\theta}} &= 0
\end{aligned}
$$

To review the derivation of the Euler-lagrange equations from the principle of least action I found a [good video](https://www.youtube.com/watch?v=sFqp2lCEvwM). Based on the equations for kinetic and potential energy above, we can now calculate all the relevant partial derivatives. 

$$
\begin{aligned}
\frac{\partial L}{\partial x} &= 0 \\

\frac{\partial L}{\partial \dot{x}} &= (M + m) \dot{x} + ml \dot{\theta} \cos{\theta} \\

\frac{\partial L}{\partial \theta} &=  - m l \dot{x} \theta \sin{\theta} + m g l \sin(\theta) = m l \sin{\theta} (g - \dot{x} \theta) \\

\frac{\partial L}{\partial \dot{\theta}} &= m l^2 \dot{\theta} + m l \dot{x} \cos{\theta} \\

\end{aligned}
$$

Now plugging those partials into the euler-lagrange equations we ultimately get: (Still need to work these out after my massive fuck up. )

$$
\begin{aligned}
- (M + m) \ddot{x} - m l \ddot{\theta} \cos{\theta} + m l \dot{\theta}^2 \sin{\theta} &= F \\
g \sin{\theta} - l \ddot{\theta} - \ddot{x} \cos{\theta} &= 0

\end{aligned}
$$

We can manipulate the above equations to solve for $\ddot{\theta}$ and $\ddot{x}$. Doing that we get 

$$
\begin{aligned}
\ddot{x} &= \frac{-F - mg \cos{\theta} \sin{\theta} + ml \dot{\theta}^2 \sin{\theta}}{M + m \sin^2{\theta}} \\

\ddot{\theta} &= \frac{F \cos{\theta} - m l \theta^2 \sin{\theta} \cos{\theta} + (M + m) g \sin{\theta}}{Ml + ml \sin^2{\theta}}
\end{aligned}
$$

### Linearization of the problem
To simply the problem further, we make the assumption of small angles and therefore are able to linearize the above equations. 

$$
\begin{aligned}
-(M + m) \ddot{x} - ml \ddot{\theta} &= F \\
g \theta - l \ddot{\theta} - \ddot{x} = 0
\end{aligned}
$$

With minimal algebraic manipulation we can rewrite these equations as:

$$
\begin{aligned}
\ddot{x} &= - \frac{F}{M} - \frac{mg\theta}{M} \\

\ddot{\theta} &= \frac{mg}{Ml} \theta + \frac{g}{l} \theta + \frac{F}{Ml}
\end{aligned}
$$

We can now write everything out as four first order first-order differential equations. 


### Applying a Linear Quadratic Regulator with infinite time horizons to solve the controls problem
We will define our state vector like so
$$
\mathbf{x} = \begin{bmatrix} x_1 \ x_2 \ x_3 \ x_4 \end{bmatrix} = \begin{bmatrix} x \ \dot{x} \ \theta \ \dot{\theta} \end{bmatrix} \\
$$

We will define our input vector as 
$$
\mathbf{u} = \begin{bmatrix} F \end{bmatrix}
$$

The first order system of differential equations can then be expressed in matrix form:
$$
\dot{\mathbf{x}} = \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u}
$$

Where A is defined like so: 
$$
\mathbf{A} = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 0 & 0 & - \frac{mg}{M} & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & \frac{mg}{Ml} + \frac{g}{l} & 0 \end{bmatrix}
$$

and B is defined like so
$$
\mathbf{B} = \begin{bmatrix} 0 \ - \frac{1}{M} \ 0 \ \frac{1}{Ml} \end{bmatrix}
$$

One method we can use to solve this time-invarient linear system is Linear Quadratic Regulators (LQR) which we can use to find the optimal control input by minimizing the quadratic cost function: 

$$
\int_0^\infty (x^T Q x + u^T R u) dt
$$

where $q_1$, $q_2$, $q_3$, $q_4$, and $r$ are non-negative weights 

$$
\mathbf{Q} = \begin{bmatrix} q_1 & 0 & 0 & 0 \\ 0 & q_2 & 0 & 0 \\ 0 & 0 & q_3 & 0 \\ 0 & 0 & 0 & q_4 \end{bmatrix}

\mathbf{R} = [r]
$$

The above cost function is quadratic, and we can therefore a solution is provided by the [linear-quadratic regular (LQR)](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator). 

Specifically, the control law that minimizes the cost function is:

$$
K = - R^{-1} (B^TP) \tilde{x}
$$

Where P is found by solving the [algebraic Riccati equation](https://en.wikipedia.org/wiki/Algebraic_Riccati_equation).

$$
\dot{P}(t) = P A + A^T P + Q - PBR^{-1} B^T P 
$$

Because this problem has infinite time horizons, $\dot{P} = 0$. $P$ therefore is a constant matrix that satisfies:

$$
P A + A^TP + Q - PBR^{-1} B^T P = 0
$$

The evolution of the optimal state vector $\tilde{x}$ can then be described. 
$$
\dot{\tilde{x}} = (A - B R^{-1}B^TP)\tilde{x}
$$





In [283]:
import numpy as np
import scipy

def get_matrices(M, m, g, l, q1, q2, q3, q4, r):
    """
    Returns the system matrix A and input matrix B.

    Parameters:
    M (float): Mass of the cart.
    m (float): Mass of the pendulum.
    g (float, optional): Acceleration due to gravity. 
    l (float, optional): Length of the pendulum.
    q1 (float): relative weight of position of cart.
    q2 (float): relative weight of velocity of cart. 
    q3 (float): relative weight of displacement theta of pole.
    q4 (float): relative weight of angular velocity of pole. 
    r (float): relative weight of control force. 


    Returns:
    A (numpy.ndarray): System matrix.
    B (numpy.ndarray): Input matrix.
    R (numpy.ndarray): weights of control vars.
    Q (numpy.ndarray): weights of system vars. 
    """

    A = np.array([
        [0, 1, 0, 0],
        [0, 0, (m * g) / M, 0],
        [0, 0, 0, 1],
        [0, 0, (m * g) / (M * l) + g / l, 0]
    ])

    B = np.array([
        [0],
        [1 / M],
        [0],
        [1 / (M * l)]
    ])

    R = np.array([r])

    Q = np.array([
        [q1, 0, 0, 0],
        [0, q2, 0, 0],
        [0, 0, q3, 0],
        [0, 0, 0, q4]
    ])

    return A, B, Q, R


def find_P(A, B, Q, R):
    """
    Return the matrix P, which satisfies the Riccati differential equation.

    Parameters:
    -----------
    A (numpy.ndarray): System matrix (4,4).
    B (numpy.ndarray): Input matrix (4,1).
    Q (numpy.ndarray): Weights of system vars (4, 4).
    R (numpy.ndarray): Weights of control vars (1, 1). 

    Returns
    -------
    P           : the matrix solution of the Riccati equation. 
    """

    print(f"A shape {A.shape}")
    print(f"Q shape {Q.shape}")

    def func(P_flat):
        P = np.reshape(P_flat, (4, 4))

        result = P @ A + A.T @ P + Q - P @ B*np.linalg.inv(R) @ B.T * P

        return np.reshape(result, 16)
    

    P = scipy.linalg.solve_continuous_are(A, B, Q, R)

    print("OUTPUT:")
    print(P)

    return P


def solve_cart_control(t, X0, A, B, R, Q, P):
    """
    Given an initial condition, solve the control problem at each time step. 

    Parameters: 
    -----------
    t (numpy.ndarray): time values with shape (n+1, )
    X0 (numpy.ndarray): Initial conditions on state variables (4,1)
    A (numpy.ndarray): System matrix (4,4).
    B (numpy.ndarray): Input matrix (4,1).
    Q (numpy.ndarray): Weights of system vars (4, 4).
    R (numpy.ndarray): Weights of control vars (1, 1). 
    P (numpy.ndarray): Matrix that solves Riccati DE (4, 4).

    Returns
    -------
    X (numpy.ndarray): the state vector at each time (n+1, 4). 
    U (numpy.ndarray): the control values (n+1, 1). 

    """

    X = np.zeros((t.size, 4))
    U = np.zeros((t.size, 1))
    X[0] = X0

    prev_t_val = 0
    for i, t_val in enumerate(t):
        delta_t = t_val - prev_t_val
        prev_t_val = t_val 

        u = -1/R @ B.T @ P @ X[i]

        print("U should by 1 by 4 I believe")
        print(u.shape)
        print("inverse r size")
        print((1/R).shape)
        print("b transpose shape")
        print((B.T).shape)

        print("product shape")
        print((B.T @ P).shape)

        U[i] = u

        if i == 0:
            X[i] = X0
        else:
            X[i] = X[i-1].T + delta_t * (A @ X[i-1].T + B * u)

    return X, U

In [109]:
M, m = 17, 2
l = 4
g = 9.8
q1, q2, q3, q4 = 1, 1, 1, 1
r = 10

A, B, Q, R = get_matrices(M, m, g, l, q1, q2, q3, q4, r)
P = find_P(A, B, Q, R)

X0 = np.array([-1, -1, 0.1, -0.2])
t = np.linspace(0, 10, 100)

X, U = solve_cart_control(t, X0, A, B, R, Q, P)

A shape (4, 4)
Q shape (4, 4)
OUTPUT:
[[ 1.22201499e+01  7.41660312e+01 -7.99956548e+02 -5.11699006e+02]
 [ 7.41660312e+01  8.24691796e+02 -9.26388989e+03 -5.92652565e+03]
 [-7.99956548e+02 -9.26388989e+03  5.23350152e+05  3.19964740e+05]
 [-5.11699006e+02 -5.92652565e+03  3.19964740e+05  1.95724664e+05]]
U should by 1 by 4 I believe
()
inverse r size
(1,)
b transpose shape
(1, 4)
product shape
(1, 4)
U should by 1 by 4 I believe
()
inverse r size
(1,)
b transpose shape
(1, 4)
product shape
(1, 4)


AttributeError: attribute 'T' of 'numpy.ndarray' objects is not writable